# TorchScript Language Reference
This reference manual describes the syntax and core semantics of the TorchScript Language. 
TorchScript is a statistically typed subset of the Python language. 

TorchScript focuses specifically on the features of Python that are needed to represent neural network models in PyTorch.

# 1. Terminology 
TorchScript and this script and has many Terminologies
______________________________________________________________________________________________________________________
```
Pattern                                                         Notes

 1. ::=                                        Indicates that the given symbol is defined as.

 
 2. " "                               Represents real keywords and delimiters that are part of the syntax.


3. A | B                                                   Indicates either A or B.


4. ( )                                                       Indicates grouping.


5. []                                                       Indicates optional.


6. A+                              Indicates a regular expression where term A is repeated at least once.


7. A*                              Indicates a regular expression where term A is repeated zero or more times.
```
---------------------
# 2. Type System
TorchScript is a statically typed subset of Python. The largest difference between TorchScript and the full Python language is that TorchScript only supports a small set of types that are needed to express neural net models.

-------------------------
# 3. TorchScript Types
The TorchScript type system consists of <code>TSType</code> and <code>TSModuleType</code>
<b>
```python
TSAllType ::= TSType | TSModuleType
TSType ::= TSMetalType | TSPrimitiveType | TSStructuralType | TSNominalType
```
</b>
<code>TSType</code> represents the majority of TorchScript types that are composable and that can be used in TorchScript type annotaions.

It referes to this:-
- Meta Types like <code>Any</code>
- Primitive Types like <code>Int</code>, <code>float</code>, <code>str</code>
- Structural Types like <code>Optional[int]</code> or <code>List[MyClass]</code>
- Nominal Types (Classes) like <code>CustomClass</code> (user-defined), <code>torch.tensor</code> (builtin)

<code>TSModuleType</code> represents ```torch.nn.Module``` and its subclasses. It is treated differently from <code>TSType</code> because of its type schema is inferred partly from the object instance and partly from the class definition.

As such, instances of ```TSModuleType``` may not follow the same static type schema. <code>TSModuleType</code> cannot be used as a TorchScript type annotation or be composed with ```TSType``` for safety considerations.

## Meta Types
These are so abstract that they are more like type constraints than concrete types.

TorchScript currently supports ```Any``` meta type which represents any TorchScript type.
### <code>Any</code> type
The ```Any``` type has no type constraints so there is no type-checking on ```Any```. As such it can be bound to any Python or TorchScript data types like ```int```, ```tuple```.
<b>
```python
TSMetaType ::= "Any"
```
</b>
1. Meta Type <code>Any</code> is the Python class from the <code>typing</code> module so you need to import it use it like this

```python
from typing import Any
```
2. The set of operators allowed on this type <code>Any</code> is also limited like
    - Assignment to data if <code>Any</code> type.
    - Binding to parameter or return of <code>Any</code> type.
    - x is, it is not where x is of <code>Any</code> type.
    - <code>isinstance(x, Type)</code> where x is of <code>Any</code> type.
    - Data of <code>Any</code> type is printable.
    - Data of <code>List[Any]</code> type may be sortable if the data is a list of values of the same type <code>T</code> and that <code>T</code> supports comparison operators.


3. <code>Any</code> is like the <code>Object</code> class in Python only that it supports a subset of the operators and methods of <code>Object</code>.

### Design Notes
During scripting a PyTorch module, we may encounter data which is not involved in the execution of the script.
But still, it has to be described by a type schema otherwise it may lead to scripting failures.

<code>Any</code> is introduced to describe the type of data where precise static types are not necessary for compilation.

<b>Example 1</b>

It illustrates how <code>Any</code> can be used to allow the second element of the <code>tuple</code> parameter to be of any type.
    
It is possible because <code>x[1]</code> is not involved in any compilation that requires knowing its precise type.

In [2]:
import torch

from typing import Tuple, Any

@torch.jit.export
def inc_first_element(x: Tuple[int, Any]):
    return (x[0] + 1, x[1])


m = torch.jit.script(inc_first_element)
print(m((1, 2.0)))
print(m((1, (100, 200))))

(2, 2.0)
(2, (100, 200))


The second element of the <code>tuple</code> is of <code>Any</code> type, thus can bind to multiple types.

For example, ```(1, 2.0)``` binds a ```float``` type to ```Any``` as in ```Tuple[int, Any]```

### Example 2
It illustrates how we can use <code>isinstance</code> to dynamically check the type of the data that is annotated as 
<code>Any</code> type.

In [3]:
def f(a: Any):
    print(a)
    return (isinstance(a, torch.Tensor))

ones = torch.ones([2])
m = torch.jit.script(f)
print(m(ones))

 1
 1
[ CPUFloatType{2} ]
True


## Primitive Types
Primitive TorchScript types are types that represent a single type of value and go with a single pre-defined type name.
<b>
```python

TSPrimitiveType ::= int | float | double | complex | bool | str | None 
```
</b>

## Structural Types
Structural types are types that are structurally defined without using a user-defined name (unlike in nominal types), 
such as <code>Future[int]</code>.

Structural types are composable with any <code>TSType</code>.
<b>
```python
TSStructuralType ::= TSTuple | TSNamedTuple | TSList | TSDict | TSOptional | TSFuture | TSRRef
 
TSTuple          ::=  "Tuple" "[" (TSType ",")* TSType "]"
TSNamedTuple     ::=  "namedTuple" "(" (TSType ",")* TSType ")"
TSList           ::=  "List" "[" TSType "]"
TSOptional       ::=  "Optional" "[" TSType "]"
TSFuture         ::=  "Future" "[" TSType "]"
TSRRef           ::=  "RRef" "[" TSType "]"
TSDict           ::=  "Dict" "[" KeyType "," TSType "]"
KeyType          ::=  str | int | float | bool | TensorType | Any
```
</b>

- ```Tuple```, ```List```, ```Optional```, ```Future```, ```Dict``` represent Python type class names that are defined in the module <code>typing</code>. So, to use these names import them from ```typing```.

```python
from typing import Tuple, List, Optional, Future, Dict
```

- ```namedTuple``` represents the Python class <code>collections.namedtuple</code> or ```typing.NamedTuple```.
- ```Future``` and ```RRef``` represent the Python classes ```torch.futures``` and ```torch.distributed.rpc```.


The TorchScript Structural types often support a common subset of the operators and methods of their Python counterparts.

### Example 1

This Example uses ```typing.NamedTuple``` syntax to define a ```tuple```.

In [5]:
from typing import NamedTuple, Tuple

class MyTuple(NamedTuple):
    first: int
    second: int
    
def inc(x: MyTuple) -> Tuple[int, int]:
    return (x.first + 1, x.second + 1)

t = MyTuple(first=1, second=2)
scripted_inc = torch.jit.script(inc)
print('TorchScript: ', scripted_inc(t))

TorchScript:  (2, 3)


### Example 2

This example uses ```collections.namedtuple``` syntax to define a ```tuple```.

In [6]:
from collections import namedtuple

_AnnotatedNamedTuple = NamedTuple('_NamedTupleAnnotated', [('first', int), ('second', int)])
_UnannotatedNamedTuple = namedtuple('_NamedTupleAnnotated', ['first', 'second'])

def inc(x: _AnnotatedNamedTuple) -> Tuple[int, int]:
    return (x.first + 1, x.second + 1)

m = torch.jit.script(inc)
print(inc(_UnannotatedNamedTuple(1, 2)))

(2, 3)


### Example 3

This example illustrates a common mistake of annotating structural types. which is not importing composite type classes 
from the ```typing``` module.

In [7]:
# ERROR: Tuple not recognized because not imported from typing 
@torch.jit.export
def inc(x: Tuple[int, int]):
    return (x[0] + 1, x[1] + 1)

m = torch.jit.script(inc)
print(m((1, 2)))

(2, 3)


## Nominal Types 

Nominal TorchScript types are Python classes. These types are called nominal because they are declared with a custom name and are compared using class names.

Nominal classes are further classified into the following categories
<b>
```python
TSNominalType    ::=  TSBuiltinClasses  |  TSCustomClass | TSEnum 
```
</b>

Among them, ```TSCustomClass``` and ```TSEnum``` must be compilable to TorchScript Representation (IR) which is enforced by the Type Checker.

## Built-In Class

Built-in nominal types are Python Classes whose semantics are built into the TorchScript System like ```Tensor types```.

TorchScript defines the semantics of these built-in nominal types, and often supports only a subset of the methods or attributes of its Python class definition.
<b>
```python
TSBuiltinClass     ::=  TSTensor | "torch.device" | "torch.Stream" | "torch.dtype" | 
                        "torch.nn.ModuleList" | "torch.nn.ModuleDict" | ...

TSTensor           ::=  "torch.Tensor" | "common.SubTensor" | "common.SubWithTorchFunction" | 
                        "torch.nn.parameter.Parameter" | subclasses of torch.Tensor
```
</b>

### Special Note on <code>torch.nn.ModuleList</code> and <code>torch.nn.ModuleDict</code>

Although ```torch.nn.ModuleList``` and ```torch.nn.ModuleDict``` is completely unrolled so that elements of
```torch.nn.ModuleList``` or keys of ```torch.nn.ModuleDict``` can be different subclasses of ```torch.nn.Module```.

### Example

This Example highlights the use of a few built-in TorchScript classes (<code>torch.*</code>).

```python
@torch.jit.script
class A:
    def __init__(self):
        self.x = torch.rand(3)
    
    def f(self, y: torch.device):
        return self.x.to(device=y)

def g():
    a = A()
    return a.f(torch.device('cuda'))

script_g = torch.jit.script(g)
print(script_g.graph)

```

In [24]:
# See in Code/TorchScript_builtin_Class.py
%run Code/BuiltInClass/TorchScript_builtin_Class.py

graph():
  %15 : Device = prim::Constant[value="cuda"]()
  %a.1 : __torch__.A = prim::CreateObject()
  %1 : NoneType = prim::CallMethod[name="__init__"](%a.1) # D:\ML\PyTorch\PyTorch_Manual\TorchScript Language Reference\Code\BuiltInClass\TorchScript_builtin_Class.py:12:8
  %5 : Tensor = prim::CallMethod[name="f"](%a.1, %15) # D:\ML\PyTorch\PyTorch_Manual\TorchScript Language Reference\Code\BuiltInClass\TorchScript_builtin_Class.py:13:11
  return (%5)



## Custom Class 
Unlike built-in classes, semantics of custom classes are user-defined and the entire class definition must be compilable to TorchScript IR and subject to TorchScript type-checking rules.
<b>
```python
TSClassDef    ::=  [@torch.jit.script]
                    "class" ClassName [(object)] :
                        MethodDefinition | 
                    [@torch.jit.ignore] | [@torch.jit.unused]
                        MethodDefinition
```
</b>

- Classes must be new-style classes. Python 3.x supports only new-style classes.

- Instance Data attributes are statically typed, and instance attributes must be declared by assignments inside the ```__init__()``` method.

- Method overloading is not supported i.e you cannot have multiple methods with the same method name.

- ```MethodDefinition``` must be compatible to TorchScript IR and adhere to TorchScript's type checking rules (i.e all methods must be valid TorchScript Functions and class attribute definitions must be valid TorchScript statements).

- ```torch.jit.ignore``` and ```torch.jit.unused``` can be used to ignore the method or function that is not fully torchscriptable or should be ignored by the compiler.

#### Compare to Python 3.x
TorchScript custom classes are quite limited compared to their Python counterpart. 
1. They do not support class attributes.
2. They do not support subclassing except for subclassing an interface type of object.
3. They do not support Method Overriding.
4. Must initialize all its instance attributes in ```__init__()```, because TorchScript constructs a static schema of the class by inferring attribute types in ```__init__()```.
5. Must contain only methods that satisfy TorchScript type-checking rules and are compilable to TorchScript IRs.

### Example 1

Python classes can be used in TorchScript if they are annotated with ```@torch.jit.script```, similar to how TorchScript function would be declared:

```python
@torch.jit.script
class MyClass:
    def __init__(self, x: int):
        self.x = x
    
    def inc(self, val: int):
        self.x += val 
```

In [15]:
%run Code/CustomClass/Example1.py
# It Works......!!!

### Example 2 

A TorchScript custom class type must "declare" all its instance attributes by assignments in ```__init__()```. If an instance attribute is not defined ```__init__()``` but accessed in other methods of the class, the class cannot be compiled as a TorchScript Class, as shown
```python
import torch

@torch.jit.script
class foo:
    def __init__(self):
        self.y = 1

# ERROR: self.x is not defined in __init__()
def assign_x(self):
    self.x = torch.rand(2, 3)
```

The class would fail to compile 
```python
RuntimeError:
Tried to set nonexistent attribute: x. Did you forget to initialize it in __init__()?:
def assign_x(self):
    self.x = torch.rand(2, 3)
    ~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
```


### Example 3

In this, a TorchScript Custom class defines a class variable name, which is not allowed
```python
import torch

@torch.jit.script
class MyClass(object):
    name = "MyClass"
    def __init__(self, x: int):
        self.x = x

def fn(a: MyClass):
    return a.name
```



It will lead to this error:
```python
RuntimeError:
'__torch__.MyClass' object has no attribute or method 'name'. Did you forget to initialize an attribute in __init__()?:
    File "test-class2.py", line 10
def fn(a: MyClass):
    return a.name
        ~~~~~~ <--- HERE
```


## Enum Type 

Like custom classes, semantics of the enum type are user-defined and the entire class definition must be compilable to TorchScript IR and adhere to TorchScript type-checking rules.
<b>
```python
TSEnumDef      ::=  class Identifier (enum.Enum | TSEnumType) :
                    ( MemberIdentifier = Value ) + 
                    ( MethodDefinition )*
```
</b>

#### Where:
- Value must be a TorchScript literal of type ```int```, ```float``` or ```str``` and must be of the same TorchScript type.
- ```TSEnumType``` is the name of a TorchScript enumerated type. Similar to Python ```enum```, TorchScript allows restricted ```Enum``` subclassing that is, subclassing an enumerated is allowed only if it does not define any members.

#### Compared To Python
- TorchScript supports only ```enum.Enum```. It does not support other variations such as ```enum.IntEnum```, ```enum.Flag```, ```enum.IntFlag``` and ```enum.auto```.
- Values of TorchScript enum members must be of the same type and can only be ```int```, ```float``` or ```str``` types, whereas Python ```enum``` members can be of any type.
- Enums containing methods are ignored by TorchScript.

### Example 1 

This Example defines the class ```Color``` as an ```Enum``` type:
```python
import torch
from enum import Enum

class Color(Enum):
    RED = 1
    GREEN = 2

def enum_fn(x: Color, y: Color) -> bool:
    if x == Color.RED:
        return True
    return x == y

m = torch.jit.script(enum_fn)

print("Eager: ", enum_fn(Color.RED, Color.GREEN))
print("TorchScript: ", m(Color.RED, Color.GREEN))
```

In [26]:
%run Code/EnumType/Example1.py # Working..!!

Eager:  True
TorchScript:  True


### Example 2

This Example shows the case of restricted enum subclassing, where ```BaseColor``` does not interfere any member, thus can be subclassed by ```Color```:
```python
import torch
from enum import Enum

class BaseColor(Enum):
    def foo(self):
        pass

class Color(BaseColor):
    RED = 1
    GREEN = 2
    
def enum_fn(x: Color, y: Color) -> bool:
    if x == Color.RED:
        return True
    return x == y

m = torch.jit.script(enum_fn)

print("TorchScript: ", m(Color.RED, Color.GREEN))
print("Eager: ", enum_fn(Color.RED, Color.GREEN))

```

In [27]:
%run Code/EnumType/Example2.py

TorchScript:  False
Eager:  True


## TorchScript Module Class

```TSModuleType``` is a special class type that is inferred from object instances that are created outside TorchScript.

```TSModuleType``` is named by the Python class of the ```object``` instance. The ```__init__()``` method of the Python class is not considered a TorchScript method, so it does not comply with TorchScript's type-checking rules.

The Type schema of the module instance class is constructed directly from an instance object (created outside the scope of TorchScript) rather than inferred from ```__init__()``` like custom classes. It is possible that two objects of the same instance class type follow two different type schemas.

In this sense, ```TSModuleType``` is not really a static type. Therefore, for type safety considerations, ```TSModuleType``` cannot be used in a TorchScript type annotation or be composed with ```TSType```.

## Module Instance Class

TorchScript module type represents the type schema of a user-defined by PyTorch module instance. When scripting a PyTorch module, the module object is always created outside TorchScript (i.e passed in as parameter to ```forward```).

The Python module class is treated as a module instance class, so ```__init__()``` method of Python module class is not subject to type-checking rules of TorchScript.

```python
    
    
TSModuleType    ::=  class Identifier (torch.nn.Module) :
                        ClassBodyDefinition
```
#### Where
- ```forward()``` and other methods decorated with ```@torch.jit.export``` must be compilable to TorchScript IR and subject to TorchScript's type checking rules.

Unlike Custom classes, only the ```forward``` method and other methods decorated with ```@torch.jit.export``` of the module type need to be compilable.

Most notably ```__init__()``` is not considered a TorchScript method. Module Type Constructors cannot be invoked within the scope of TorchScript.

Instead, TorchScript module objects are always constructed outside and passed into ```torch.jit.script(ModuleObj)```.

### Example 1
This Example illustrates a few features of module types

1. The ```TestModule``` instance created outside the scope of TorchScript.
2. ```__init__()``` is not considered a TorchScript method so it does not have to be annotated and can contain arbitrary Python code. In addition, the ```__init__()``` method of an instance class cannot be invoked in TorchScript code because the ```TestModule```instances are instantiated in Python
3. In this Example, ```TestModule(2.0)``` and ```TestModule(2)``` create two instances with different types for its data attributes. ```self.x```is of type ```float``` for ```TestModule(2.0)```, whereas ```self.y``` is of type ```int``` for ```TestModule(2.0)```.
4. TorchScript automatically compiles other methods like ```mul()``` invoked by methods annotated via ```@torch.jit.export``` or ```forward()``` methods.
5. Entry points to a TorchScript program are either ```forward()``` of a module type, functions annotated as ```torch.jit.script```, or methods annotated as ```torch.jit.export```. 

```python
# This Example illustrates a few features of module types
import torch

class TestModule(torch.nn.Module):
    def __init__(self, v):
        super().__init__()
        self.x = v
    
    def forward(self, inc: int):
        return self.x + inc


m = torch.jit.script(TestModule(1))
print(f'First Instance : {m(3)}')

m = torch.jit.script(TestModule(torch.ones([5])))
print(f"Second Instance: {m(3)}")
```

In [28]:
%run Code/ModuleInstanceClass/Example1.py

First Instance : 4
Second Instance: tensor([4., 4., 4., 4., 4.])


### Example 2

This Example shows an incorrect usage of module type. Specifically, this example invokes the constructor of ```TestModule``` inside the scope of TorchScript.
```python
# This Example shows an incorrect usage of module type. Specifically, this example invokes the constructor of TestModule inside the scope of TorchScript.
import torch

class TestModule(torch.nn.Module):
    def __init__(self, v):
        super().__init__()
        self.x = v
    
    def forward(self, x: int):
        return self.x + x

class MyModel:
    def __init__(self, v: int):
        self.val = v
    
    @torch.jit.export
    def do(self, val: int) -> int:
        # ERROR: should not invoke the constructor of module type
        model = TestModule(self.val)
        return model(val)

m = torch.jit.script(MyModel(2)) # Results in below RuntimeError
# RuntimeError: Could not get name of python class object

```

In [31]:
# ERROR: RuntimeError: Could not get name of python class object

#%run Code/ModuleInstanceClass/Example2.py 

# 4. Type Annotation

Since TorchScript is statically typed, programmers need to annotate types at <i>straight points</i> of TorchScript code so that every local variable or instance data attribute has a static type, and every function and method has a statically typed signature

## When to Annotate Types
Type annotations are only needed in places where static types cannot be automatically inferred. Types of local variables and data attributes are often automatically inferred from their assignment statements. Sometimes an inferred type may be too restrictive e.g:- ```x``` being inferred as ```NoneType``` through assignment ```x = None ```, 
whereas ```x``` is actually used as an ```Optional```. 

In such cases, type annotations may be needed to overwrite auto inference eg:- ```x: Optional[int] = None```.
Note that it is always safe to type annotate a local variable or data attribute even if its type can be automatically inferred. The annotated type must be congruent with TorchScript’s type-checking.

When a parameter, local variable, or data attribute is not type annotated and its type cannot be automatically inferred, TorchScript assumes it to be a default type of ```TensorType```, ```List[TensorType]``` or ```Dict[str, TensorType]```.

## Annotate Function Signature

Since a parameter may not be automatically inferred from the body of the function (including both functions and methods), they need to be type annotated. Otherwise, they assume the default type ```TensorType```.

TorchScript supports two styles for method and function signature type annotation
- <b>Python3 Style</b> :- It annotates types directly on the signature. As such it allows individual parameters to be left unannotated (whose type will be default type of ```TensorType```) or allows the return type to be left unannotated (whose type will be automatically inferred)

<b>
    
```python
Python3Annotation    ::=  def Identifier [ ( ParamAnnot* ) ] [ ReturnAnnot ] :
                                FuncOrMethodBody
   
ParamAnnot           ::=  Identifier[ : TSType ] , 
ReturnAnnot          ::=  ->  TSType
```
</b>

Note, when using <b>Python3</b> style, the type ```self``` is automatically inferred and should not be annotated.

- <b>Mypy Style</b> :- It annotates types as a comment right below the function/method declaration. In the Mypy style, since parameter names does not appear in the annotation, all parameters have to be annotated

<b>
    
```python
MyPythonAnnotation    ::= "# type:" "(" ParamAnnot* ")"  [ReturnAnnot]
ParamAnnot            ::= TSType  ","
ReturnAnnot           ::= ->  TSType    
```
</b>

### Example 1

In this example 
- ```a``` is not annotated and assumes the default type of ```TensorType```.
- ```b``` is annotated as type ```int```.
- The return  type is not annotated and is automatically inferred as type ```TensorType```. based on the type of value being returned).

```python

# Using Python3 style.
import torch

def f(a, b: int):
    return a + b

m = torch.jit.script(f)
print('TorchScript: ', m(torch.ones([6]), 100))

```

In [32]:
%run Code/TypeAnnotation/AnnoteFunctionSignature/Example1.py

TorchScript:  tensor([101., 101., 101., 101., 101., 101.])


### Example 2

The Following example uses ```Mypy``` style annotation. Note that parameters or return values must be annotated even if some of them assume the default type.

In [33]:
import torch 

def f(a, b):
    # type: (torch.Tensor, int) -> torch.Tensor
    return a + b

m = torch.jit.script(f)
print('TorchScript: ', m(torch.ones([6]), 100))

TorchScript:  tensor([101., 101., 101., 101., 101., 101.])


## Annotate Variables and Data Attributes

In general, the types of data attributes (including class and instance data attributes) and local variables can be automatically inferred from assignment statements. Sometimes, however, if a variable or attribute is associated with values of different types (e.g as ```None``` or ```TensorType```), then may need to be explicitly type annotated as a <i>wider</i> type such as ```Optional[T]``` and ```Any```

### Local Variables 

Local variables can be annotated according to Python3 typing module annotation rules i.e
<b>
```python
LocalVarAnnotation    ::=  Identifier  [":" TSType]  "=" Expr 
```
</b>

In general, types of local variables can be automatically inferred. In some cases, however, you may need to annotate a mulit-type for local variables that may be associated with different concrete types.

Typical multi-types include ```Optional[T]``` and ```Any```.
#### Example

In [34]:
import torch
from typing import Optional

def f(a, setVal: bool):
    value: Optional[torch.Tensor] = None
    if setVal:
        value = a
    return value

ones = torch.ones([6])
m = torch.jit.script(f)
print('TorchScript: ', m(ones, True), m(ones, False))

TorchScript:  tensor([1., 1., 1., 1., 1., 1.]) None


### Instance Data Attributes

For ```ModuleType``` classes, instance data attributes can be annotated according to Python 3.x ```typing``` module annotation rules. 

Instance data attributes can be annotated as final via ```Final```
<b>
```python
class ClassIdentifier  (torch.nn.Module)
InstanceAttrIdentifier  :  ["Final ("] TSType [")"]
...
```
</b>
<b>Where</b> :-

- ```InstanceAttrIdentifier``` is the name of an instance attribute.
- ```Final``` indicates that the attribute cannot be re-assigned outside of ```__init__``` or overriden in subclasses.

#### Example

In [35]:
import torch

class MyModule(torch.nn.Module):
    offset_: int

def __init__(self, offset):
    self.offset_ = offset

...

Ellipsis

## Type Annotation APIs

```torch.jit.annotate(T, expr)```

This API annotates type ```T``` to an expression ```expr```. This often used when the default type of an expression is not the type intended by the programmer. 

For example, an empty list (dir) has the default type ```List[TensorType]``` (```Dict[TensorType, TensorType]```), but sometimes it may be used to initialize a list of some types.

Another common use case is for annotating the return type of ```tensor.tolist()```. However, it cannot be used to annotate the type of a module attribute in <i>```__init__```</i>;

```torch.jit.Attribute``` should never be used for this instead.
### Example 

In this example, ```[]``` is declared as a list of integers via ```torch.jit.annotate``` (instead of assuming ```[]``` to be the default type of ```List[TensorType]```)

In [36]:
import torch
from typing import List

def g(l: List[int], val: int):
    l.append(val)
    return l

def f(val: int):
    l = g(torch.jit.annotate(List[int], []), val)
    return l


m = torch.jit.script(f)
print("Eager: ", f(3))
print("TorchScript: ", m(3))

Eager:  [3]
TorchScript:  [3]


## Type Annotation Appendix

### TorchScript Type System Definition

<b>
    
```python
TSAllType          ::=  TSType | TSModuleType
TSType             ::=  TSMetaType | TSPrimitiveType | TSStructuralType | TSNominalType
 
TSMetaType         ::=  Any
TSPrimitiveType    ::=  int | float | double | complex | bool | str | None
    
TSStructuralType   ::=  TSTuple | TSNamedTuple | TSList | TSDict | 
                        TSOptional | TSFuture | TSRRef

TSTuple            ::=  Tuple [ (TSType ",")* TSType ]
TSNamedTuple       ::=  namedTuple ( (TSType ",")* TSType )
TSList             ::=  List [ TSType ]
TSOptional         ::=  Optional [ TSType ]
TSFuture           ::=  Future [ TSType ]
TSRRef             ::=  RRef [ TSType ] 
TSDict             ::=  Dict [ KeyType , TSType ]
KeyType            ::=  str | int | float | bool | TensorType | Any

TSNominalType      ::=  TSBuiltInClasses | TSCustomClass | TSEnum
TSBuiltInClass     ::=  TSTensor | torch.device | torch.stream |
                        torch.dtype | torch.nn.ModuleList | torch.nn.ModuleList | ...

TSTensor           ::=  torch.tensor and subclasses
```
</b>

---------------------

# 5. Expressions

It is modeled after [Expressions chapter of Python Language Reference](https://docs.python.org/3/reference/expressions.html)

## Arithmetic Conversions

There are number of implicit type conversions that are performed in TorchScript:
- A ```Tensor``` with a ```float``` and ```int``` data type can be implicitly converted to an instance of ```FloatType``` or ```IntType``` provided that it has a size of 0, does not have ```require_grad``` set to ```True``` and does not require narrowing.
- Instances of ```StringType``` can be implicitly converted into ```DeviceType```.
- The implicit conversion rules from the above two points can be applied to instances of ```TupleType``` to produce instances of ```ListType``` with appropriate contained type.

Explicit conversions can be invoked using the ```float```, ```int```, ```bool``` and ```str``` built-in functions that accept primitive data types as arguments and can accepts user-defined types if they implement ```__bool__```, ```__str__```.

## Atoms

Atoms are the most basic elements of expression
<b>
```python
atom       ::=  identifier | literal | enclosure
enclosure  ::=  parenth_form | list_display | dict_display
```
</b>

## Identifiers

The rules that dictate what is a legal identifer in TorchScript are the same as their [Python Counterparts](https://docs.python.org/3/reference/lexical_analysis.html#identifiers)

## Literals 

<b>
    
```python
literal   ::=  stringliteral | integer | floatnumber 
```
</b>

Evaluation of a literal yields an object of the appropriate type with the specific value (with approximations applied as necessary for floats). 

Literals are immutable, and multiple evaluations of identical literals may obtain the same object or distinct objects with the same value.[stringliteral](https://docs.python.org/3/reference/lexical_analysis.html#string-and-bytes-literals), [integer](https://docs.python.org/3/reference/lexical_analysis.html#integer-literals), and [floatnumber](https://docs.python.org/3/reference/lexical_analysis.html#floating-point-literals) are defined in the same way as their Python counterparts.

## Paranthesized Forms

<b>
    
```python
parenth_form   ::=  '(' [expression_list] ')'
```
</b>

A paranthesized expression list yields whatever the expression list yields. If the list contains at least one comma, it yields a ```Tuple```, otherwise it yields the single expression inside the expression list.

An empty pair of parantheses yields an empty ```Tuple``` object (```Tuple[]```).

## List and Dictionary Displays

<b>
    
```python 
list_comprehension    ::=  expression comp_for
comp_for              ::=  for target_list in or_expr 
list_display          ::=  '[' [expression_list | list_comprehension] ']'
dict_display          ::=  '{' [key_datum_list | dict_comprehension] '}'
key_datum_list        ::=  key_datum (',' key_datum)*
key_datum             ::=  expression ':' expression
dict_comprehension    ::=  key_datum comp_for
```
</b>

Lists and dicts can be constructed by either listing the container contents explicitly or by providing instrunctions on how to compute them via a set of looping instructions i.e <b><i>comprehension</i></b>.

A comprehension is semantically equivalent to using a for loop and appending to an ongoing list. Comprehensions implicitly create their own scope to make sure that the items of the target list do not leak into the enclosing scope. In the case that container items are explicitly listed, the expressions in the expression list are evaluated left-to-right.

If a key is repeated in a ```dict_display``` that has a ```key_datum_list```, the resultant dict uses the value from the rightmost datum in the list that uses the repeated key.

## Primaries
<b>
    
```python
    
primary    ::=  atom | attributeref | subscription | slicing | call
```
</b>

### 1. Attribute References
<b>
    
```python
attributeref   ::=  primary '.' identifier
```
</b>

The ```primary``` must evaluate to an object of a type that supports attribute references that have an attribute named ```identifier```.

### 2. Subscriptions
<b>

```python
subscription   ::=  primary '[' expression_list ']'
```
</b>

The ```primary``` must evaluate to an object that supports subscription:
- If primary is a ```List```, ```Tuple``` or ```str``` the expression list must evaluate to an integer or slice.
- If the primary is a ```Dict```, the expression list must evaluate to an object of the same type as key type ```Dict```.
- If the primary is ```ModuleList```, the expression list must be an ```integer``` literal.
- If the primary is ```ModuleDict```, the expression must be a ```stringliteral```.

### 3. Slicings

A slicing selects a range of items in a ```str```, ```List``` or ```Tensor```. Slicings may be used as expressions or targets in assignment or ```del``` statements.

<b>
    
```python
slicing      ::=  primary [ slice_list ] 
slice_list   ::=  slice_item ( ',' slice_item)* [',']
slice_items  ::=  expression | proper_slice
proper_slice ::=  [expression] ':' [expression] [':' [expression] ]
```
</b>

### 4. Calls

<b>
    
```python
call          ::=  primary '(' argument_list ')'
argument_list ::=  args [',' kwargs] | kwargs
args          ::=  [arg (',' arg)*]
kwargs        ::=  [kwarg (',' kwarg)*]
kwarg         ::=  arg '=' expression
arg           ::=  identifier
```
</b>

The ```primary``` must desugar or evaluate to a callable object. All argument expressions are evaluted before the call is attempted.

## Power Operator
<b>
    
```python
power ::=  primary ['**' u_expr]
```
</b>

The power operator has the same semantics as the built-in ```pow``` function. It binds more tightly than unary operators on the left, but less tightly than unary operators on the right. i.e ```-2 ** 3 == -(2 ** (-3))```.
The left and right operators can be ```int```, ```float``` or ```Tensor```.

Scalers are broadcast in the case of scaler-tensor/tensor-scaler exponentiation operations, and tensor-tensor exponentiation is done elementwise without any broadcasting.

## Unary and Arithmetic Bitwise Operations

<b>
    
```python
u_expr   ::=  power | '-' power | '~' power
```
</b>

The unary ```-``` operator yields the negation of its argument. The unary ```~``` operator yields the bitwise inversion of its argument. ```-``` can be used with ```int```, ```float```, and ```Tensor``` of ```int``` and ```float```. ```~``` can only be used with ```int``` and ```Tensor``` of ```int```.

## Binary Arithmetic Operations

<b>
    
```python
m_expr ::=  u_expr | m_expr '*' u_expr | m_expr '@' m_expr | m_expr '//' u_expr | m_expr '/' 
            u_expr | m_expr '%' u_expr
a_expr ::=  m_expr | a_expr '+' m_expr | a_expr '-' m_expr
```
</b>

The binary arithmetic operators can operate on ```Tensor```, ```int```, and ```float```. For tensor-tensor ops, both arguments must have the same shape. For scalar-tensor or tensor-scalar ops, the scalar is usually broadcast to the size of the tensor.

Division ops can only accept scalars as their right-hand side argument, and do not support broadcasting.

The ```@``` operator is for matrix multiplication and only operates on ```Tensor``` arguments. The multiplication operator ```(*)``` can be used with a list and integer in order to get a result that is the original list repeated a certain number of times.

## Shifting Operations

<b>
    
```python
shift_expr    ::=  a_expr | shift_expr ( '<<' | '>>' ) a_expr
```
</b>

These operators accept two ```int``` arguments, two ```Tensor``` arguments, or a ```Tensor``` argument and an ```int``` or ```float``` argument.
In all cases, a right shift by ```n``` is defined as floor division by ```pow(2, n)```, and a left shift by ```n``` is defined as multiplication by ```pow(2, n)```.

When both arguments are ```Tensors```, they must have the same shape. When one is scaler and the other is a ```Tensor```, the scaler is logically broadcast to match the size of the ```Tensor```.


## Binary Bitwise Operations

<b>
    
```python
and_expr ::=  shift_expr | and_expr '&' shift_expr
xor_expr ::=  and_expr | xor_expr '^' and_expr
or_expr  ::=  xor_expr | or_expr '|' xor_expr
```
</b>

The ```&``` operator computes the bitwise ```AND``` of its arguments, the ```^``` the bitwise ```XOR```, and the ```|``` the bitwise ```OR```. Both operands must be ```int``` or ```Tensor```, or the left operand must be ```Tensor``` and the right operand must be ```int```.

When both operands are ```Tensor```, they must have the same shape. When the right operand is ```int```, and the left operand is ```Tensor```, the right operand is logically broadcast to match the shape of the ```Tensor```.

## Comparisons

<b>

```python
comparison    ::=  or_expr (comp_operator or_expr)*
comp_operator ::=  < | > | == | >= | <= | != | is [not] | [not] in
```
    
</b>
    
### Value Comparisons
The operators ```<, >, ==, >=, <=, and !=``` compare the values of two objects. The two objects generally need to be of the same type, unless there is an implicit type conversion available between the objects. 
    
User-defined types can be compared if rich comparison methods (e.g., ```__lt__```) are defined on them. Built-in type comparison works like Python:
- Numbers are compared mathematically
- Strings are compared lexicographically
- ```lists```, ```tuples```, and ```dicts``` can be compared only to other ```lists```, ```tuples```, and ```dicts``` of the same type and are compared using the comparison operator of corresponding elements.
    
### Membership Test Operations
                                  
The operators ```in``` and ```not in``` test for membership. ```x in s``` evaluates to ```True```. If ```x``` is a member of ```s``` and ```False``` otherwise x ```not in s``` is equivalent to ```not x in s```.
                                  
This operator is supported for ```lists```, ```dict``` and ```tuples``` and can be used with user-defined types if they implement the ```__contains__``` method.
                                  
### Identity Comparisons
         
For all types except ```int```, ```double```, ```bool``` and ```torch.device``` operators ```is``` and ```is not``` test  for the object's identity; ```x is y``` is ```True```. 

if and and only if ```x``` and ```y``` are the same object. For all other types, ```is``` is equivalent to comparing them using ```==```. ```x is not y``` yields the inverse of ```x is y```.
   
## Boolean Operations

<b>
    
```python
or_test  ::=  and_test | or_test 'or' and_test
and_test ::=  not_test | and_test 'and' not_test
not_test ::=  'bool' '(' or_expr ')' | comparison | 'not' not_test
```
</b>

User-defined objects can customize their conversion to ```bool``` by implementing a ```__bool__``` method. The operator not yields ```True``` if its operand is false, ```False``` otherwise.
  
## Conditional Expressions

<b>
   
```python
conditional_expression ::=  or_expr ['if' or_test 'else' conditional_expression]
expression            ::=  conditional_expression
```
</b>

The expression ```x if c else y``` first evaluates the condition ```c``` rather than ```x```. If ```c``` is ```True```, ```x``` is evaluated and its value is returned; otherwise, ```y``` is evaluated and its value is returned. As with if-statements, ```x``` and ```y``` must evaluate to a value of the same type.
  
## Expression Lists

<b>
    
```python
expression_list ::=  expression (',' expression)* [',']
starred_item    ::=  '*' primary
```
</b>
    
A starred item can only appear on the left-hand side of an assignment statement, e.g., ```a, *b, c = ...```
  
------------------------
    
# 6.Simple Statements
 
The following section describes the syntax of simple statements that are supported in TorchScript. Referred from [Simple Statements Python Language](https://docs.python.org/3/reference/simple_stmts.html).
 
## Expression Statements
 
<b>

```python
expression_stmt    ::=  starred_expression
starred_expression ::=  expression | (starred_item ",")* [starred_item]
starred_item       ::=  assignment_expression | "*" or_expr
```
</b>
    
## Assignment Statements

<b>

```python
assignment_stmt ::=  (target_list "=")+ (starred_expression)
target_list     ::=  target ("," target)* [","]
target          ::=  identifier
                     | "(" [target_list] ")"
                     | "[" [target_list] "]"
                     | attributeref
                     | subscription
                     | slicing
                     | "*" target
    
```
</b>
 
## Augmented Assignment Statements

<b>
    
```python
augmented_assignment_stmt ::= augtarget augop (expression_list)
augtarget                 ::= identifier | attributeref | subscription
augop                     ::= "+=" | "-=" | "*=" | "/=" | "//=" | "%=" |
                              "**="| ">>=" | "<<=" | "&=" | "^=" | "|="
```
</b>
    
## Annotated Assignment Statements

<b>
  
```python
annotated_assignment_stmt ::= augtarget ":" expression
                              ["=" (starred_expression)]
```
</b>
    
## The ```raise``` Statement

<b> 
   
```python 
  
raise_stmt  ::=  raise [expression [from expression]]
```
</b>
    
## Other Simple Statements
 
<b>
 
```python
assert_stmt ::=  assert expression ["," expression]

return_stmt ::=  return [expression_list]
    
del_stmt ::=  del target_list
 
pass_stmt ::= pass
  
print_stmt ::= print "(" expression  [, expression] [.format{expression_list}] ")"
   
break_stmt ::= break
  
continue_stmt  ::=  continue
```
</b>

----------------------------------
 
# 7. Compound Statements

This section is referred from [Compound Statements Python Language](https://docs.python.org/3/reference/compound_stmts.html)
    
## The ```if``` Statement
 
TorchScript supports both basic and ternery ```if/else```.
 
### a. Basic ```if/else``` Statement
 
<b>

```python
    
if_stmt   ::=  if assignment_expression : suite
               (elif assignment_expression : suite)
               [else : suite]
```
</b>
    
### b. Ternary ```if/else``` Statement

<b>
 
```python
if_stmt  ::=  return [expression_list] if assignment_expression else [expression_list]
```
</b>
    
### Example 1
 
A ```Tensor``` with 1 dimension is promoted to ```bool```

In [37]:
import torch

@torch.jit.script
def fn(x: torch.Tensor):
    if x: # If Tensor is promoted
        return True
    return False

print(fn(torch.rand(1)))

True


### Example 2

A ```Tensor``` with multi dimensions are not promoted to ```bool```

In [38]:
import torch
# Multi dimensional Tensors are error out

@torch.jit.script
def fn():
    if torch.rand(2):
        print('Tensor is available')

    if torch.rand(4, 5, 6):
        print('Tensor is available')

print(fn()) # It is expected...!!

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "<ipython-input-38-85ad910462d9>", line 6, in fn
@torch.jit.script
def fn():
    if torch.rand(2):
       ~~~~~~~~~~~~ <--- HERE
        print('Tensor is available')
RuntimeError: Boolean value of Tensor with more than one value is ambiguous


### Example 3

In this example, only the ```True``` branch is evaluated, since ```a``` is annotated as ```final``` and set to ```True```.

In [39]:
import torch

a : torch.jit.final[Bool] = True

if a:
    return torch.empty(2, 3)
else:
    return []

AttributeError: module 'torch.jit' has no attribute 'final'

### c. The ```while``` Statement

<b>
    
```python
while_stmt  ::=  while assignment_expression : suite
```
</b>

<i>while…else</i> statements are not supported in Torchscript. It results in a ```RuntimeError```

### d. The ```for-in``` Statement

<b>
    
```python
for_stmt  ::= for target_list in expression_list : suite
                [else : suite]
```
</b>

```for...else``` statements are not supported in Torchscript. It results in a ```RuntimeError```.

### Example 1

For loops on tuples: these unroll the loop, generating a body for each member of the tuple. The body must type-check correctly for each member.

In [41]:
import torch
from typing import Tuple

@torch.jit.script
def fn():
    tup = (3, torch.ones(4))
    for x in tup:
        print(x)
        
fn()

3
 1
 1
 1
 1
[ CPUFloatType{4} ]


### Example 2 

For loops on lists; for loops over a ```torch.nn.ModuleList``` will unroll the body of the loop at compile time, with each member of the module list.

In [44]:
# For loops on lists; for loops over a torch.nn.ModuleList will unroll the body of the loop at compile time, 
# with each member of the module list.

import torch

class SubModule(torch.nn.ModuleList):
    def __init__(self):
        super(SubModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.randn(2))
    
    def forward(self, input):
        return self.weight + input
    
class MyModule(torch.nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.mods = torch.nn.ModuleList([SubModule() for i in range(10)])
    
    def forward(self, v):
        for module in self.mods:
            v = module(v)
        return v

model = torch.jit.script(MyModule())

model

RecursiveScriptModule(
  original_name=MyModule
  (mods): RecursiveScriptModule(
    original_name=ModuleList
    (0): RecursiveScriptModule(original_name=SubModule)
    (1): RecursiveScriptModule(original_name=SubModule)
    (2): RecursiveScriptModule(original_name=SubModule)
    (3): RecursiveScriptModule(original_name=SubModule)
    (4): RecursiveScriptModule(original_name=SubModule)
    (5): RecursiveScriptModule(original_name=SubModule)
    (6): RecursiveScriptModule(original_name=SubModule)
    (7): RecursiveScriptModule(original_name=SubModule)
    (8): RecursiveScriptModule(original_name=SubModule)
    (9): RecursiveScriptModule(original_name=SubModule)
  )
)

### e. The ```with``` Statement
 
The ```with``` statement is used to wrap the execution of a block with methods defined by a context manager

<b>
 
```python
with_stmt  ::=  with with_item (',' with_item) : suite
with_item  ::=  expression [as target]
```
</b>

- If a target was included in the ```with``` statement, the return value from the context manager's ```__enter__()``` is assigned to it.
- Unlike in Python, if an exception caused to be exited, its type, value and traceback are not passed as arguments to ```__exit__()```. Three ```None``` arguments are supplied.
- ```try```, ```except``` and ```finally``` statements are not supported inside ```with``` blocks.
- Exceptions raised within ```with``` block cannot be suppressed.

### f. The ```tuple``` Statement

<b>
 
```python
tuple_stmt  ::=  tuple([iterables])
```
</b>

- Iterables types in TorchScript include ```Tensor```, ```List```, ```Tuple```, ```dict```, ```String```, ```torch.nn.ModuleList``` and ```torch.nn.ModuleDict```
- You cannot convert a List to Tuple by using this built-in function.

Unpacking all outputs into a tuple is covered by:
```python
abc = func() # Function that returns a Tuple
a, b = func()
```

### g. The ```getattr``` Statement

<b>
    
```python
getattr_stmt  ::=  getattr(object, name[, default])
```
</b>

- Attribute name must be a literal string.
- Module type object is not supported (e.g., torch._C).
- Custom class object is not supported (e.g., torch.classes.*).

### h. The ```hasattr``` Statement

<b>
    
```python
hasattr_stmt  ::=  hasattr(object, name)
```
</b>

- Attribute name must be a literal string.
- Module type object is not supported (e.g., torch._C).
- Custom class object is not supported (e.g., torch.classes.*).

### i. The ```zip``` Statement

<b>
    
```python
zip_stmt  ::=  zip(iter1, iter2)
```
</b>

### j. The ```enumerate``` Statement

<b>
    
```python
enumerate_stmt  ::=  enumerate([iter])
```
</b>

- Arguments must be iterables.
- Iterable types in TorchScript include ```Tensors, lists, tuples, dictionaries, strings, torch.nn.ModuleList and torch.nn.ModuleDict```.